In [2]:
import torch
import torchvision.models as models

from bnn import BConfig, prepare_binary_model
# Import a few examples of quantizers
from bnn.ops import BasicInputBinarizer, BasicScaleBinarizer, XNORWeightBinarizer

model = models.resnet18(weights='ResNet18_Weights.DEFAULT')
bconfig = BConfig(
    activation_pre_process = BasicInputBinarizer,
    activation_post_process = BasicScaleBinarizer,
    # optionally, one can pass certain custom variables
    weight_pre_process = XNORWeightBinarizer.with_args(center_weights=True)
)
bmodel = prepare_binary_model(model, bconfig)

from torchsummary import summary
summary(bmodel, (3, 32, 32), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
BasicInputBinarizer-1            [-1, 3, 32, 32]               0
XNORWeightBinarizer-2              [-1, 3, 7, 7]               0
BasicScaleBinarizer-3           [-1, 64, 16, 16]               0
            Conv2d-4           [-1, 64, 16, 16]           9,408
       BatchNorm2d-5           [-1, 64, 16, 16]             128
              ReLU-6           [-1, 64, 16, 16]               0
         MaxPool2d-7             [-1, 64, 8, 8]               0
BasicInputBinarizer-8             [-1, 64, 8, 8]               0
XNORWeightBinarizer-9             [-1, 64, 3, 3]               0
BasicScaleBinarizer-10             [-1, 64, 8, 8]               0
           Conv2d-11             [-1, 64, 8, 8]          36,864
      BatchNorm2d-12             [-1, 64, 8, 8]             128
             ReLU-13             [-1, 64, 8, 8]               0
BasicInputBinarizer-14          

In [7]:
import hls4ml

config = hls4ml.utils.config_from_pytorch_model(bmodel, granularity='name')
hls_model = hls4ml.converters.convert_from_pytorch_model(bmodel,(3, 32, 32), hls_config=config, output_dir='binary_proj')
hls_model.write()

Interpreting Model ...
Topology:
Input Shape:  [[3, 32, 32]]
Layer name: conv1, layer type: Conv2D, input shape: [[3, 32, 32]]


Exception: Unsupported layer BasicInputBinarizer